# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import sys
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list[0])

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect() # To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)

#### Create Keyspace

In [3]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [5]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(artist_name text, song text, length decimal, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [6]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (artist_name, song, length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        data = (line[0], line[9], float(line[5]), int(line[8]), int(line[3]))
        session.execute(query, data)

#### Do a SELECT to verify that the data have been inserted into each table

In [7]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select1 = "SELECT artist_name, song, length, session_id, item_in_session FROM music_library WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(select1)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song, row.length, row.session_id, row.item_in_session)

Faithless Music Matters (Mark Knight Dub) 495.3073 338 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [8]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(artist_name text, song text, item_in_session int, session_id int, first_name text, last_name text, user_id int, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_library (artist_name, song, item_in_session, session_id, first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        data = (line[0], line[9], int(line[3]), int(line[8]), line[1], line[4], int(line[10]))
        session.execute(query, data)

## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select1 = "SELECT artist_name, song, item_in_session, session_id, first_name, last_name, user_id FROM artist_library WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(select1)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song, row.item_in_session, row.session_id, row.first_name, row.last_name, row.user_id)

Down To The Bone Keep On Keepin' On 0 182 Sylvie Cruz 10
Three Drives Greece 2000 1 182 Sylvie Cruz 10
Sebastien Tellier Kilometer 2 182 Sylvie Cruz 10
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 182 Sylvie Cruz 10


In [9]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS history_library "
query = query + "(first_name text, last_name text, song text, PRIMARY KEY (song, first_name, last_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO history_library (first_name, last_name, song)"
        query = query + " VALUES (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        data = (line[1], line[4], line[9])
        session.execute(query, data)

## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select1 = "SELECT first_name, last_name, song FROM history_library WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(select1)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name, row.song)
                    

Jacqueline Lynch All Hands Against His Own
Sara Johnson All Hands Against His Own
Tegan Levine All Hands Against His Own


### Drop the tables before closing out the sessions

In [10]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table history_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [11]:
session.shutdown()
cluster.shutdown()